In [1]:
import torch
from torchmetrics import F1Score
import flash
from flash.image import ImageClassificationData, ImageClassifier

In [2]:
import os
rootdir = './Dev/vinted_dev_train'
class_dict = {}
for count, file in enumerate(os.listdir(rootdir)):
    class_dict[file] = count
    
print(class_dict)

{'women__clothes__coats_and_jackets__coats__duster_coats': 0, 'women__clothes__coats_and_jackets__coats__parkas': 1, 'women__clothes__coats_and_jackets__coats__oversized_coats': 2}


In [37]:
import inspect

print(inspect.signature(ImageClassificationData.from_folders))

(train_folder: Union[str, NoneType] = None, val_folder: Union[str, NoneType] = None, test_folder: Union[str, NoneType] = None, predict_folder: Union[str, NoneType] = None, train_transform: ~INPUT_TRANSFORM_TYPE = <class 'flash.image.classification.input_transform.ImageClassificationInputTransform'>, val_transform: ~INPUT_TRANSFORM_TYPE = <class 'flash.image.classification.input_transform.ImageClassificationInputTransform'>, test_transform: ~INPUT_TRANSFORM_TYPE = <class 'flash.image.classification.input_transform.ImageClassificationInputTransform'>, predict_transform: ~INPUT_TRANSFORM_TYPE = <class 'flash.image.classification.input_transform.ImageClassificationInputTransform'>, target_formatter: Union[flash.core.data.utilities.classification.TargetFormatter, NoneType] = None, input_cls: Type[flash.core.data.io.input.Input] = <class 'flash.image.classification.input.ImageClassificationFolderInput'>, transform_kwargs: Union[Dict, NoneType] = None, **data_module_kwargs: Any) -> 'ImageClas

In [3]:
datamodule = ImageClassificationData.from_folders(
    train_folder="Dev/vinted_dev_train/",
    val_folder="Dev/vinted_dev_val/",
    batch_size=4,
    transform_kwargs={"image_size": (100, 100), "mean": (0.485, 0.456, 0.406), "std": (0.229, 0.224, 0.225)},
)

In [5]:
# 2. Build the task
f1 = F1Score(num_classes=3)
model = ImageClassifier(backbone="mobilenetv3_small_100", labels=datamodule.labels, metrics=f1)

/home/nemish/.local/lib/python3.8/site-packages/pytorch_lightning/utilities/parsing.py:261: UserWarning: Attribute 'metrics' is an instance of `nn.Module` and is already saved during checkpointing. It is recommended to ignore them using `self.save_hyperparameters(ignore=['metrics'])`.
  rank_zero_warn(
Using 'mobilenetv3_small_100' provided by rwightman/pytorch-image-models (https://github.com/rwightman/pytorch-image-models).
No pretrained weights exist for this model. Using random initialization.


In [6]:
# 3. Create the trainer and finetune the model

trainer = flash.Trainer(max_epochs=3, gpus=torch.cuda.device_count())
trainer.finetune(model, datamodule=datamodule, strategy="freeze")

GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs

  | Name          | Type           | Params
-------------------------------------------------
0 | train_metrics | ModuleDict     | 0     
1 | val_metrics   | ModuleDict     | 0     
2 | test_metrics  | ModuleDict     | 0     
3 | adapter       | DefaultAdapter | 1.5 M 
-------------------------------------------------
15.2 K    Trainable params
1.5 M     Non-trainable params
1.5 M     Total params
6.084     Total estimated model params size (MB)


Sanity Checking: 0it [00:00, ?it/s]

[W NNPACK.cpp:51] Could not initialize NNPACK! Reason: Unsupported hardware.


Training: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

In [12]:
#datamodule = ImageClassificationData.from_folders(predict_folder="Dev/vinted_dev_val/women__clothes__coats_and_jackets__coats__parkas", batch_size=4)

In [7]:
# 4. Predict what's on a few images!
predict_module = ImageClassificationData.from_files(
    predict_files=[
        "Dev/vinted_dev_val/women__clothes__coats_and_jackets__coats__oversized_coats/1704261785.jpeg",
        "Dev/vinted_dev_val/women__clothes__coats_and_jackets__coats__oversized_coats/1778451667.jpeg",
        "Dev/vinted_dev_val/women__clothes__coats_and_jackets__coats__parkas/1657095323.jpeg",
    ],
    batch_size=4,
)


In [69]:
print(model.labels)

['women__clothes__coats_and_jackets__coats__duster_coats', 'women__clothes__coats_and_jackets__coats__oversized_coats', 'women__clothes__coats_and_jackets__coats__parkas']


In [8]:
predictions = trainer.predict(model, datamodule=predict_module, output="labels")
print(predictions)

Predicting: 179it [00:00, ?it/s]

[['women__clothes__coats_and_jackets__coats__duster_coats', 'women__clothes__coats_and_jackets__coats__oversized_coats', 'women__clothes__coats_and_jackets__coats__duster_coats']]


### Sklearn Classification Report

In [9]:
from sklearn.metrics import classification_report

# =============================================================================
y_true = [
    "women__clothes__coats_and_jackets__coats__oversized_coats",
    "women__clothes__coats_and_jackets__coats__oversized_coats",
    "women__clothes__coats_and_jackets__coats__parkas",
]
#module_labels = model.labels
#print(module_labels)

y_pred = np.array(predictions[0])
print("================================================")
print(
    classification_report(
        y_true,
        y_pred,
        labels=datamodule.labels,
        zero_division=0,  # turn off warnings
    )
)


                                                           precision    recall  f1-score   support

   women__clothes__coats_and_jackets__coats__duster_coats       0.00      0.00      0.00         0
women__clothes__coats_and_jackets__coats__oversized_coats       0.67      1.00      0.80         2
         women__clothes__coats_and_jackets__coats__parkas       0.00      0.00      0.00         1

                                                micro avg       0.67      0.67      0.67         3
                                                macro avg       0.22      0.33      0.27         3
                                             weighted avg       0.44      0.67      0.53         3



### Torch metrics F1 Score

In [10]:
predictions_trans = []
predictions_trans = [class_dict[x] for x in predictions[0] if x in class_dict.keys()]

print(predictions_trans)

[2, 2, 2]


In [11]:
target = torch.tensor([1, 1, 2])
preds = torch.tensor(predictions_trans)
f1 = F1Score(num_classes=3, average='macro')
f1(preds, target)

tensor(0.2500)

In [36]:
dir(ImageClassificationData)


['__annotations__',
 '__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__jit_unused_properties__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_get_property',
 '_load_model_on_after_batch_transfer_fns',
 '_predict_dataloader',
 '_reset_iterator',
 '_resolve_dataloader_collate_fn',
 '_resolve_on_after_batch_transfer_fn',
 '_set_hparams',
 '_show_batch',
 '_split_train_val',
 '_test_dataloader',
 '_to_hparams_dict',
 '_train_dataloader',
 '_val_dataloader',
 'add_argparse_args',
 'configure_data_fetcher',
 'data_fetcher',
 'dims',
 'from_argparse_args',
 'from_csv',
 'from_data_frame',
 'from_datasets',
 'from_fiftyone',
 'from_files',
 'from_folders',
 'from_labelstudio',
 'from_numpy',
 'from_tensors',
 'get_ini

In [84]:
print(inspect.signature(classification_report))

(y_true, y_pred, *, labels=None, target_names=None, sample_weight=None, digits=2, output_dict=False, zero_division='warn')
